In [ ]:
import os
import json
import traceback

# ★ New: Builderの窓口関数だけをインポート
from src.bridge.builder import build_all_configs 

# Engine関連
from src.schema.definitions import OutlookConfig
from src.adapter.outlook import OutlookAdapter
from src.engine.core import GenericEtlEngine

# =================================================================
# Step 2: Bridgeの実行 (Manager)
# =================================================================
# 以前の run_bridge_compile 関数は削除！
# 理由: 詳細なコンパイル手順は src/bridge/builder.py に隠蔽したため。

# =================================================================
# Step 3: Engineの実行 (JSON -> Run)
# =================================================================
def run_engine(json_path):
    print(f"3️⃣  [Engine] Starting Bot...")
    
    if not os.path.exists(json_path):
        print(f"❌ Config not found: {json_path}")
        return

    try:
        with open(json_path, "r", encoding="utf-8") as f:
            config_data = json.load(f)
            # Pydanticモデルでバリデーション
            config = OutlookConfig(**config_data)

        adapter = OutlookAdapter()
        engine = GenericEtlEngine(config, adapter)
        engine.run()
        
    except Exception as e:
        print(f"❌ Engine Error: {e}")
        traceback.print_exc()

# =================================================================
# Main Execution
# =================================================================
if __name__ == "__main__":
    
    # -------------------------------------------------------------
    # Phase 1: Bridge (コンパイル)
    # -------------------------------------------------------------
    print("2️⃣  [Bridge] Compiling All Specs...")
    
    # ★ ここが変わった！
    # ファイルパスも、何個ファイルがあるかも気にせず、
    # 「とにかく全部の設計書をJSONにしてくれ」と頼むだけ。
    build_all_configs() 

    print("-" * 50)

    # -------------------------------------------------------------
    # Phase 2: Engine (実行)
    # -------------------------------------------------------------
    # Engineには「メインの設定ファイル」の場所だけ教えればOK
    # (mail_business_rules.jsonの場所は、このJSONの中に書いてあるから)
    MAIN_JSON_PATH = "configs/accounting/invoice_bot_v2.json"
    
    run_engine(MAIN_JSON_PATH)